##### Conducting some research on BWM and Audi offerings and tiers

Based on findings on its respective websites ([BMW](https://www.bmw.lt/lt/all-models.html) and [Audi models](https://www.audi.lt/lt/web/lt/models.html)), I'll reduce all models to the following hierarchy seen below.

![](images/full-hierarchy.png)

__N.B.__ M series under the BWM appears twice because this is how BWM themselves structure their offerings on the website.

TODO: REVISIT THIS!!!

1. This hierarchy is missing X Series breakdown (X2, X5, X6)
2. Separation of concerns apply to ML as well (This is to justify the need for modularity w.r.t. ML models)
3. Rename top bar to VIN

The diagram above is not 100 % accurate depiction of the reality as this requires much deeper investigation and is a bit too time consuming to deal with it properly right now.

For the sake of simplicity and reduce this task to the following idea is:
 * Each VIN can be mapped to make (e.g. BWM, Audi, etc)
 * Each make can be mapped to a series or model (e.g. 8 series, A8, etc)
 * Each model will have a specific body (e.g. Coupe, Sedan, etc)
 * Each model

![](images/simple-hierarchy.png)

####  Label preprocessing

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/data_validation/vin_model_pairs_good.csv")
df = df[["vin", "model"]].drop_duplicates()
df

,vin,model
0,3MW5R1J0XM8CXXXXX,3 Series
1,4USBT33443LRXXXXX,Z4
2,5UXCR4C06M9FXXXXX,X5
3,5UXCY6C04P9PXXXXX,NaN
4,5UXCY6C04P9PXXXXX,X6
...,...,...
2030,WUAZZZF5XJA9XXXXX,A5
2031,WUAZZZF5XJA9XXXXX,NaN
2032,WUAZZZFX9J79XXXXX,R8
2035,WUAZZZGY4NA9XXXXX,NaN


In [3]:
df.groupby("model")["vin"].agg("count").reset_index().rename(columns={"vin": "count"})

,model,count
0,1 Series,74
1,2 Series,25
2,3 Series,257
3,335,1
4,4,17
5,4 Series,5
6,5 Series,213
7,530D (EUR),1
8,6 Series,6
9,7 Series,36


In [4]:
# TODO: Move this to a config file
bmw_3_series = ["335"]
bmw_4_series = ["4", "428I (USA)"]
bmw_5_series = ["530D", "530D (EUR)", "M5"]
bmw_6_series = ["630dx (630dx)", "640dx (640dx)"]
audi_a5 = ["S5"]
audi_a7 = ["rs 7"]
audi_q5 = ["SQ5"]

In [5]:
# TODO: Refactor this into something less embarrassing
bmw_3_series_dict = dict.fromkeys(bmw_3_series, "3 Series")
bmw_4_series_dict = dict.fromkeys(bmw_4_series, "4 Series")
bmw_5_series_dict = dict.fromkeys(bmw_5_series, "5 Series")
bmw_6_series_dict = dict.fromkeys(bmw_6_series, "6 Series")
audi_a5_dict = dict.fromkeys(audi_a5, "A5")
audi_a7_dict = dict.fromkeys(audi_a7, "A7")
audi_q5_dict = dict.fromkeys(audi_q5, "Q5")

d1 = {**bmw_3_series_dict,
     **bmw_4_series_dict,
     **bmw_5_series_dict,
     **bmw_6_series_dict,
     **audi_a5_dict,
     **audi_a7_dict,
     **audi_q5_dict,
     **audi_q5_dict,
}

display(d1)

{'335': '3 Series',
 '4': '4 Series',
 '428I (USA)': '4 Series',
 '530D': '5 Series',
 '530D (EUR)': '5 Series',
 'M5': '5 Series',
 '630dx (630dx)': '6 Series',
 '640dx (640dx)': '6 Series',
 'S5': 'A5',
 'rs 7': 'A7',
 'SQ5': 'Q5'}

In [6]:
df["updated_model"] = df["model"].map(d1).fillna(df["model"])
display(df)

,vin,model,updated_model
0,3MW5R1J0XM8CXXXXX,3 Series,3 Series
1,4USBT33443LRXXXXX,Z4,Z4
2,5UXCR4C06M9FXXXXX,X5,X5
3,5UXCY6C04P9PXXXXX,NaN,NaN
4,5UXCY6C04P9PXXXXX,X6,X6
...,...,...,...
2030,WUAZZZF5XJA9XXXXX,A5,A5
2031,WUAZZZF5XJA9XXXXX,NaN,NaN
2032,WUAZZZFX9J79XXXXX,R8,R8
2035,WUAZZZGY4NA9XXXXX,NaN,NaN


In [7]:
df.groupby("updated_model")["vin"].agg("count").reset_index().rename(columns={"vin": "count"})

,updated_model,count
0,1 Series,74
1,2 Series,25
2,3 Series,258
3,4 Series,22
4,5 Series,214
5,6 Series,6
6,7 Series,36
7,8 Series,3
8,A1,12
9,A3,72


Grouping models even more

In [8]:
bmw_x_series = ["X1", "X2", "X3", "X4", "X5", "X6",]
bmw_i = ["i3", "i8",]
bmw_z_series = ["Z3", "Z4",]

In [9]:
bmw_x_series_dict = dict.fromkeys(bmw_x_series, "X Series")
bmw_i_dict = dict.fromkeys(bmw_i, "i")
bmw_z_series_dict = dict.fromkeys(bmw_z_series, "Z Series")

d2 = {**bmw_x_series_dict,
     **bmw_i_dict,
     **bmw_z_series_dict,
}

display(d2)

{'X1': 'X Series',
 'X2': 'X Series',
 'X3': 'X Series',
 'X4': 'X Series',
 'X5': 'X Series',
 'X6': 'X Series',
 'i3': 'i',
 'i8': 'i',
 'Z3': 'Z Series',
 'Z4': 'Z Series'}

In [10]:
df["agg_model"] = df["updated_model"].map(d2).fillna(df["updated_model"])
display(df)

,vin,model,updated_model,agg_model
0,3MW5R1J0XM8CXXXXX,3 Series,3 Series,3 Series
1,4USBT33443LRXXXXX,Z4,Z4,Z Series
2,5UXCR4C06M9FXXXXX,X5,X5,X Series
3,5UXCY6C04P9PXXXXX,NaN,NaN,NaN
4,5UXCY6C04P9PXXXXX,X6,X6,X Series
...,...,...,...,...
2030,WUAZZZF5XJA9XXXXX,A5,A5,A5
2031,WUAZZZF5XJA9XXXXX,NaN,NaN,NaN
2032,WUAZZZFX9J79XXXXX,R8,R8,R8
2035,WUAZZZGY4NA9XXXXX,NaN,NaN,NaN


In [11]:
df.groupby("agg_model")["vin"].agg("count").reset_index().rename(columns={"vin": "count"})

,agg_model,count
0,1 Series,74
1,2 Series,25
2,3 Series,258
3,4 Series,22
4,5 Series,214
5,6 Series,6
6,7 Series,36
7,8 Series,3
8,A1,12
9,A3,72


##### Impute missing values

In [12]:
df["imputed_model"] = df.groupby("vin")["agg_model"].fillna(method="ffill")
df["imputed_model"] = df.groupby("vin")["imputed_model"].fillna(method="bfill")
df

,vin,model,updated_model,agg_model,imputed_model
0,3MW5R1J0XM8CXXXXX,3 Series,3 Series,3 Series,3 Series
1,4USBT33443LRXXXXX,Z4,Z4,Z Series,Z Series
2,5UXCR4C06M9FXXXXX,X5,X5,X Series,X Series
3,5UXCY6C04P9PXXXXX,NaN,NaN,NaN,X Series
4,5UXCY6C04P9PXXXXX,X6,X6,X Series,X Series
...,...,...,...,...,...
2030,WUAZZZF5XJA9XXXXX,A5,A5,A5,A5
2031,WUAZZZF5XJA9XXXXX,NaN,NaN,NaN,A5
2032,WUAZZZFX9J79XXXXX,R8,R8,R8,R8
2035,WUAZZZGY4NA9XXXXX,NaN,NaN,NaN,NaN


Some VINs had all associated model values as nulls so previous imputation logic didn't change change this, thus we can remove all VINs where model NaN.

In [13]:
final_df = df[df["imputed_model"].notnull()]
final_df = final_df[["vin", "imputed_model"]].rename(columns={"imputed_model": "model"})
final_df

,vin,model
0,3MW5R1J0XM8CXXXXX,3 Series
1,4USBT33443LRXXXXX,Z Series
2,5UXCR4C06M9FXXXXX,X Series
3,5UXCY6C04P9PXXXXX,X Series
4,5UXCY6C04P9PXXXXX,X Series
...,...,...
2028,WUAZZZF38N19XXXXX,Q3
2030,WUAZZZF5XJA9XXXXX,A5
2031,WUAZZZF5XJA9XXXXX,A5
2032,WUAZZZFX9J79XXXXX,R8


In [14]:
final_df.to_csv("../data/label_preprocessing/vin_model_pairs_w_labels.csv", index=False)